# Projekt-Dokumentation: Der Ritter der verlorenen Seelen

**Projekttitel:** Der Ritter der verlorenen Seelen

**Plattform:** Java (Swing/AWT)

**Abgabetermin:** 15.12.2025

**Modul:** Programmieren 1 - Prof. Dr. Jan Rexilius

**Gruppe:** Marvin Sandmeier, Marc Mittelstedt, Ralf Schmidt, Johann Issa

Diese Dokumentation hält den inkrementellen Aufbau des Spiels fest und kommentiert die wichtigsten architektonischen und technischen Entscheidungen für die Bereiche **(a) GUI & Leveldesign** und **(b) Spiellogik** des Aufgabenblattes.

---

## Teil 1: GUI und Leveldesign (Aufgabe a)

Dieser Abschnitt beschreibt die Implementierung der grafischen Benutzeroberfläche, der visuellen Rückmeldungen und der Levelstruktur.

### A.1. Grafische Darstellung und Zustandsverwaltung

Die Darstellung der Szene erfolgt kontinuierlich, wobei klare visuelle Rückmeldungen über den Spielzustand gegeben werden.

> **Verwendeter Prompt:**
> "Definiere in Java Swing eine Hauptklasse `GamePanel` als `JPanel` mit einem `Timer` für eine konstante Game-Loop (ca. 60 FPS). Nutze die Trennung in `paintComponent()` (Zeichnen) und `actionPerformed()` (Logik-Update) und implementiere ein Zustandssystem (`GameState`) für Menü und Spiel."

* **Designentscheidung (GUI):** Die zentrale Klasse `GamePanel` sorgt mit ihrem `Timer` (16 ms Verzögerung) für die **kontinuierlich aktualisierte Szene**. Die Zeichnung (`paintComponent`) ist strikt von der Logik (`actionPerformed`) getrennt, um eine stabile Darstellung zu gewährleisten.
* **Technische Notiz (Zustände):** Das Spiel verwendet die `GameState`-Enumeration zur Steuerung der **visuellen Zustände** ("Game Over", "Level Complete", "Menu Screen", "Spiel startet").
* **Technische Notiz (HUD):** Die Anzeige (`drawHUD`) im `GamePanel` zeigt kontinuierlich die **Punkteanzeige** und die verbleibenden Leben. Die Spielfigur und die Gegner werden als grafische Objekte bewegt und verändert.

---

### A.2. Leveldesign und Kameraführung

Die Levelstruktur unterstützt das Gameplay durch Abwechslung und eine an den Spieler angepasste Kamera.

> **Verwendeter Prompt:**
> "Implementiere eine Methode in `Level.java` zur Generierung von Leveln. Füge eine Logik für eine Kamera hinzu, die dem Spieler sanft folgt, aber an den Level-Grenzen stoppt."

* **Designentscheidung (Level-Anforderung):** Das Spiel verfügt über 8 Levels($index$ 0 bis 7). Die Levels unterscheiden sich durch Layout (Rampen, Plattformen) und enthalten neue Objekte (bewegliche Plattformen, unterschiedliche Gegner).
* **Designentscheidung (Levelwechsel):** Der **Levelwechsel** erfolgt **automatisch** (nach Erreichen der Flagge) oder durch **Spieleraktion** (Levelauswahl im Menü).
* **Technische Notiz (Kamera-System):** Die Berechnung von `camX` im `GamePanel` implementiert die Kamera-Logik. Sie zentriert das bewegte Objekt (Spielfigur), begrenzt aber die Verschiebung, um das Levelende sichtbar zu machen. Die Level wurden vollständig eigenständig designt, da das LLM Probleme hatte die Level lösbar zu gestalten. Entweder wurden nicht erreichbare Ziele, zu große Sprünge oder viel zu simple Level erzeugt.
* **Technische Notiz (Parallax):** Ein **Parallax-Scrolling-Effekt** (`parallaxShift = camX / 2`) in `Level.draw()` sorgt für visuelle Tiefe. Der Hintergrund bewegt sich also langsamer als der Boden mit den Blöcken.

---

## Teil 2: Spiellogik (Aufgabe b)

Dieser Abschnitt dokumentiert die Implementierung der Spielmechanik, der Steuerung und der objektorientierten Prinzipien.

### B.1. Steuerung und Interaktion

Die Benutzerinteraktion ist über Tastendruck umgesetzt, wobei die Eingaben eine klare Wirkung im Spiel haben.

> **Verwendeter Prompt:**
> "Erstelle einen `MenuManager`, der auf `GameState` reagiert und dynamische Buttons für Level-Auswahl generiert. Implementiere die Steuerung über WASD/Pfeiltasten mittels `InputMap` und `ActionMap`."

* **Designentscheidung (Steuerung):** Die Steuerung erfolgt über **Tastendruck** (WASD/Pfeiltasten für Bewegung und Leertaste für Springen). Die Eingaben führen zu einer **erkennbaren Wirkung** im Spiel (Bewegung, Auswahl).
* **Technische Notiz (Steuerung/Input Handling):** Die Steuerung wird über **`InputMap` und `ActionMap`** im `GamePanel` implementiert. Dies ist eine robuste, moderne Swing-Methode, die eine konfliktfreie Verarbeitung von Tasteneingaben sicherstellt.
* **Designentscheidung (Spielziel & Ablauf):** Das Spiel ist **eigenständig startbar** über das Hauptmenü ("Start"). Das einfache Ziel ist das **Sammeln von Punkten** und das Erreichen der Endflagge.

---

### B.2. Objektorientierte Prinzipien und Ereignisverarbeitung

Zur Einhaltung der OOP-Anforderungen werden klare Klassenstrukturen für Level-Elemente und Entitäten verwendet.

> **Verwendeter Prompt:**
> "Implementiere die Gegner-Kollisionslogik im `GamePanel`, wobei der Spieler nur dann Punkte erhält und abprallt (`player.bounceAfterStomp()`), wenn er fällt (`player.isFalling()`) und seine Y-Position über der des Gegners liegt."

* **Designentscheidung (OOP-Klassenhierarchie):** Die Klasse `Tile.java` dient als Basis für alle Level-Elemente. Durch **Vererbung** (`MovingPlatform` erbt von `Tile`) und **Kapselung** (`Tile.getRect()`) werden sinnvolle Klassenstrukturen verwendet und die Spiellogik von der Darstellung getrennt.
* **Designentscheidung (Ereignisverarbeitung):** Die Methode `handleEnemyCollision` im `GamePanel` erkennt und verarbeitet das **Ereignis Kollision** korrekt.
    * **Logik-Hierarchie:** Die vertikale Kollision (Stomp $\rightarrow$ **Punktestand-Änderung** und Gegner-Entfernung) wird gegenüber der horizontalen Kollision (Schaden $\rightarrow$ Leben abziehen) priorisiert.
* **Designentscheidung (Start- & Endbedingung):** Die **Startbedingung** ist der Klick auf "Start" im Menü. Die **Endbedingung** ist das Erreichen der Flagge (führt zu "Level Complete") oder 0 Leben (führt zu "Game Over").

---

### B.3. Erweiterte Logik und Persistenz

Zusätzliche Komplexität wird durch physikbasierte Objekte und ein Speichersystem zur Verfolgung des Ziels hinzugefügt.

> **Verwendeter Prompt:**
> "Schreibe eine Java-Klasse `MovingPlatform`, die sich entlang eines Vektors (Start-X/Y zu End-X/Y) bewegt. Nutze Vektorprojektion, um die Richtung bei Erreichen der Grenzen umzukehren. Implementiere eine Klasse `Storage` zum Speichern von Highscores für mehrere Levels."

* **Designentscheidung (Kollisionsvererbung):** Die `MovingPlatform` passt in ihrer `update`-Methode die geerbten `Tile`-Koordinaten (`super.x` und `super.y`) an ihre neue Fließkommaposition an. Dadurch wird sichergestellt, dass die Kollisionsbox (`getRect()`) **synchron zur Bewegung** aktualisiert wird und der Spieler korrekt mitgenommen werden kann.
* **Technische Notiz (Physik):** Die `MovingPlatform` nutzt **Vektorprojektion** zur Implementierung einer komplexen Bewegung, die eine nachvollziehbare Spielmechanik bietet. Die Gleichung der Projektion der aktuellen Position $\vec{P}$ auf die Strecke von Start $\vec{A}$ nach Ende $\vec{B}$ lautet:
  $$
  proj = \frac{(\vec{P} - \vec{A}) \cdot (\vec{B} - \vec{A})}{\|\vec{B} - \vec{A}\|^2}
  $$
* **Designentscheidung (Speicherung):** Die `Storage`-Klasse dient der **Persistenz** und verwaltet die besten Einzel-Level-Scores sowie die aggregierte `totalScore`. Dies unterstützt das einfache Ziel der Punktejagd.
* **Technische Notiz (Stabilität):** Im `GamePanel.actionPerformed` werden Kopien der Entitäten-Listen (`new ArrayList<>(enemies)`) verwendet, um Fehler bei gleichzeitiger Änderung und Iteration (z.B. Entfernung nach Stomp) zu vermeiden.